In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from scipy.optimize import curve_fit
from scipy import stats
import os

def clean_file(filepath): # Cleans the csv file to make it easier and faster to manipulate
    df = pd.read_csv(filepath, sep="\t", skip_blank_lines=True, low_memory = False, dtype = str)
    df = df.iloc[[0] + list(range(2, len(df), 2))]
    df = df.drop(df.columns[0], axis=1)
    df.to_csv("clean_file.csv", index=False)
    return 0

def lin_function(t, a, k):
    return a - k * t

def exp_function(t, a, k):
    expo = -k * t
    expo = np.clip(expo, -700, 700)
    return a * np.exp(expo)

def find_distrib(data):
    plt.hist(data, bins='scott', alpha=0.5, label='Scott')
    plt.legend()
    plt.show()
    stat, p = stats.shapiro(data)
    print('Stat=%.3f, p=%.3f' % (stat, p))
    if p > 0.05:
        print("Les données semblent suivre une distribution normale")
    else:
        print("Les données ne suivent pas une distribution normale")
    return 0

def draw_k(data):
    cycles = []
    fig, graph = plt.subplots(figsize=(20,6))
    for i in range(len(data)):
        cycles.append(i)
    graph.plot(cycles, data)
    plt.show()
    return 0

In [8]:
def project3(col_index, mini, maxi, df):
    fig = plt.figure(figsize=(20, 16))
    gs = fig.add_gridspec(3, 2, height_ratios=[1, 1, 1], width_ratios=[1, 1], hspace=0.5, wspace=0.3)
    #graph 1
    ax1 = fig.add_subplot(gs[0, :])
    x = df.iloc[0].values
    for y in df.iloc[1:].values:
        ax1.plot(x, y, color="blue", alpha=0.05)
    ax1.axvline(x[col_index], color='red', linestyle='--')
    ax1.set_ylim(0, 2)
    ax1.set_xlabel("Wavelength", fontsize=14)
    ax1.set_ylabel("Absorbance", fontsize=14)
    ax1.set_title("Absorbance over wavelength", fontsize=15)
    #graph 2
    ax2 = fig.add_subplot(gs[1, :])
    time = list(range(len(df) - 1))
    curves = df.iloc[1:, col_index].values
    ax2.plot(time, curves, color='blue')
    ax2.set_ylim(0, 2)
    ax2.set_xlabel("Time", fontsize=14)
    ax2.set_ylabel("Absorbance", fontsize=14)
    ax2.set_title(f"Absorbance over time for wavelength {df.iloc[0, col_index]:.3f}nm", fontsize=15)
    #graph 3
    ax3 = fig.add_subplot(gs[2, 0])
    time = []
    time2 = []
    time_number = 0
    for i in range(0, len(df)):
        time.append(i)
    curves = df.iloc[0:, col_index].values
    for i in range(1, len(curves)):
        if curves[i] > 0.2 and curves[i - 1] <= 0.2 and i + maxi - mini <= len(time):
            for j in range(maxi - mini):
                time2.append(i + j + mini)
            time_number += 1
    t_list = time2
    y_list = [curves[t] for t in time2]
    t = np.array(t_list)
    y = np.array(y_list)
    all_y_fit = np.zeros_like(y)
    k_list = []
    for i in range(time_number):
        start = i * (maxi - mini)
        end = start + (maxi - mini)
        t_block = t[start:end]
        y_block = y[start:end]
        t_block_norm = t_block - t_block[0]
        params, cov = curve_fit(
            exp_function, t_block_norm, y_block,
            p0=[np.max(y_block), 0.01],
            bounds=([0, 0], [np.inf, np.inf])
        )
        a_fit, k_fit = params
        residuals = y_block - exp_function(t_block_norm, a_fit, k_fit)
        chi2 = np.sum(residuals ** 2)
        ss_res = np.sum(residuals ** 2)
        ss_tot = np.sum(y_block - np.mean(y_block) ** 2)
        r_square = 1 - (ss_res / ss_tot)
        if r_square >= 0.9997225:
            k_list.append(k_fit)
        y_fit_block = exp_function(t_block_norm, a_fit, k_fit)
        all_y_fit[start:end] = y_fit_block    
    ax3.hist(k_list, bins='scott', alpha=0.5, label='Scott', histtype='stepfilled')
    stat, p = stats.shapiro(k_list)
    ax3.text(0.7, 0.9, 'Stat=%.3f, p=%.3f' % (stat, p), transform=ax3.transAxes, fontsize=12)
    if p > 0.05:
        ax3.text(0.7, 0.7, "The data appears to\nfollow a normal\ndistribution", transform=ax3.transAxes, fontsize=12)
    else:
        ax3.text(0.7, 0.7, "The data does not\nfollow a normal\ndistribution", transform=ax3.transAxes, fontsize=12)
    ax3.legend(fontsize=14)
    ax3.set_title("Distribution of k", fontsize=15)
    ax3.set_xlabel("Value of k", fontsize=14)
    ax3.set_ylabel("Frequency", fontsize=14)
    #graph 4
    ax4 = fig.add_subplot(gs[2, 1])
    cycles = []
    for i in range(len(k_list)):
        cycles.append(i)
    ax4.scatter(cycles, k_list)
    ax4.set_title("k over cycles", fontsize=15)
    ax4.set_xlabel("Number of cycles", fontsize=14)
    ax4.set_ylabel("k (s-1)", fontsize=14)
    plt.show()
    return 0

In [9]:
if not os.path.exists("clean_file.csv"):
    print("creating file...")
    clean_file("25082705_NO2BIPS_25_AcOEt 2025 août 28 08_25_02 57601.csv") # Change the filename to your actual file
    print("done")

data = pd.read_csv("clean_file.csv", header=None, sep=",") # Load the cleaned data

def safe_project(col_index, range_vals):
    min_val, max_val = range_vals
    if min_val >= max_val:
        max_val = min_val + 1
    project3(col_index, min_val, max_val, data)

wavelength_strs = data.iloc[0, :].values

options = [(int(float(val)), idx) for idx, val in enumerate(wavelength_strs)]

col_index_slider = widgets.SelectionSlider(
    options=options,
    value=651,
    description="Wavelength",
    layout=widgets.Layout(width='800px'),
)

range_slider = widgets.SelectionRangeSlider(
    options=list(range(0, 301)),
    index=(20, 120),
    description='Window',
    layout=widgets.Layout(width='800px'),
)

ui = widgets.VBox([
    col_index_slider,
    range_slider
])

out = widgets.interactive_output(
    safe_project,
    {"col_index": col_index_slider, "range_vals":range_slider}
)

display(widgets.VBox([out, ui]))

creating file...
done
